# Setup & Imports

In [3]:
import pandas as pd
import os

os.chdir('../data_files')


## Load all files into dataframes

In [23]:
print("Reading Investors.xlsx ...", end='', flush=True)
investors_df = pd.read_excel('Investors.xlsx')
print(" done.")

print("Reading Investors Account.xlsx ...", end='', flush=True)
investors_accounts_df = pd.read_excel('Investors Account.xlsx')
print(" done.")

print("Reading Managed Funds.xlsx ...", end='', flush=True)
managed_funds_df = pd.read_excel('Managed Funds.xlsx')
print(" done.")    

print("Reading Transactions with Details.xlsx ...", end='', flush=True)
transactions_with_details_df = pd.read_excel('Transactions with Details.xlsx')
print(" done.")


# For fund ops transactions (multiple files), load into a dictionary for convenience
fund_ops_files = [
    'Fund Ops Transactions 2025.xlsx',
    'Fund Ops Transactions 2024.xlsx',
     'Fund Ops Transactions 2023.xlsx',
     'Fund Ops Transactions 2022.xlsx',
     'Fund Ops Transactions 2022.xlsx',
     'Fund Ops Transactions 2021.xlsx',
     'Fund Ops Transactions 2019-2020.xlsx',
     'Fund Ops Transactions 2010-2018.xlsx'
]

fund_ops_dfs = []
for file in fund_ops_files:
    print(f"Reading {file} ...", end='', flush=True)
    df = pd.read_excel(file)
    fund_ops_dfs.append(df)
    print(" done.")

fund_ops_df = pd.concat(fund_ops_dfs, ignore_index=True)
print("All funds ops transactions loaded into a single DataFrame.")

Reading Investors.xlsx ... done.
Reading Investors Account.xlsx ... done.
Reading Managed Funds.xlsx ... done.
Reading Transactions with Details.xlsx ... done.
Reading Fund Ops Transactions 2025.xlsx ... done.
Reading Fund Ops Transactions 2024.xlsx ... done.
Reading Fund Ops Transactions 2023.xlsx ... done.
Reading Fund Ops Transactions 2022.xlsx ... done.
Reading Fund Ops Transactions 2022.xlsx ... done.
Reading Fund Ops Transactions 2021.xlsx ... done.
Reading Fund Ops Transactions 2019-2020.xlsx ... done.
Reading Fund Ops Transactions 2010-2018.xlsx ... done.
All funds ops transactions loaded into a single DataFrame.


## Inspect schemas and head rows

In [24]:
for df_name, df in {
    'investors_df': investors_df,
    'investors_accounts_df': investors_accounts_df,
    'managed_funds_df': managed_funds_df,
    'transactions_with_details_df': transactions_with_details_df,
    'fund_ops_df': fund_ops_df
}.items():
    print(f"=== {df_name} ===")
    print(df.head())
    print(df.columns)
    print(df.info())
    print("\n")


=== investors_df ===
     DB Name Investor Region                           Investor ID  \
0  HSH002788             ???  944b7a51-67ea-4e35-97ad-8c4bfce6ac7d   
1  HSH002788             ???  4e265769-2b23-42ca-9128-f5fd3eff716d   
2  HSH002788             ???  28d5995f-619d-4971-84fc-93ee9d0b4281   
3  HSH002788             ???  57132c1d-0c5d-4a15-a3c2-9482cc387ac8   
4  HSH002788             ???  9bcf0fea-17d9-4324-a877-003310a53e1e   

                    Investor Legal Name                  Investor Entity Name  \
0  944b7a51-67ea-4e35-97ad-8c4bfce6ac7d  944b7a51-67ea-4e35-97ad-8c4bfce6ac7d   
1  4e265769-2b23-42ca-9128-f5fd3eff716d  4e265769-2b23-42ca-9128-f5fd3eff716d   
2  28d5995f-619d-4971-84fc-93ee9d0b4281  28d5995f-619d-4971-84fc-93ee9d0b4281   
3  57132c1d-0c5d-4a15-a3c2-9482cc387ac8  57132c1d-0c5d-4a15-a3c2-9482cc387ac8   
4  9bcf0fea-17d9-4324-a877-003310a53e1e  9bcf0fea-17d9-4324-a877-003310a53e1e   

   Investor Assigned ID Investor First Name Investor Last Name  \
0    

## Assess joins & relatiohships

In [25]:
# check unique and non-unique IDs

print("Checking unique and non-unique IDs in each DataFrame...\n")
id_checks = [
    ("investors_df", investors_df, 'Investor ID'),
    ("investors_accounts_df", investors_accounts_df, 'Investor Account ID'),
    ("managed_funds_df", managed_funds_df, 'Fund ID'),
    ("transactions_with_details_df", transactions_with_details_df, 'Header id'),
    # ("fund_ops_df", fund_ops_df, 'Transaction ID')  
]

for name, df, col in id_checks:
    total = df[col].shape[0]
    unique = df[col].nunique()
    non_unique = total - unique
    print(f"{name} - Total: {total}, Unique: {unique}, Non-Unique: {non_unique}")


Checking unique and non-unique IDs in each DataFrame...

investors_df - Total: 4603, Unique: 4590, Non-Unique: 13
investors_accounts_df - Total: 6529, Unique: 6529, Non-Unique: 0
managed_funds_df - Total: 502, Unique: 502, Non-Unique: 0
transactions_with_details_df - Total: 387724, Unique: 173976, Non-Unique: 213748


In [26]:
# VALIDATION BLOCK: Data quick checks and integrity validation

print("=== Starting Data Validation Checks ===\n")

# 1. investors_df checks
print("Checking investors_df")

# Investor Region values
print("\nInvestor Region value counts:")
print(investors_df['Investor Region'].value_counts(dropna=False))

# Investor Legal Name vs Investor ID
investor_name_id_diff = investors_df[investors_df['Investor Legal Name'] != investors_df['Investor ID']]
print(f"\nInvestor Legal Name different from Investor ID: {len(investor_name_id_diff)} rows")

# Missing first or last names
print("\nMissing Investor First Name proportion:", investors_df['Investor First Name'].isnull().mean())
print("Missing Investor Last Name proportion:", investors_df['Investor Last Name'].isnull().mean())

# Duplicate Investor IDs
print("\nDuplicate Investor IDs:", investors_df['Investor ID'].duplicated().sum())

# ---------------------------------------------------------
# 2. investors_accounts_df checks
print("\nChecking investors_accounts_df")

# Duplicate Investor Account IDs
print("\nDuplicate Investor Account IDs:", investors_accounts_df['Investor Account ID'].duplicated().sum())

# Fund IDs validity in Managed Funds
invalid_fund_ids = investors_accounts_df[~investors_accounts_df['Fund ID'].isin(managed_funds_df['Fund ID'])]
print(f"Accounts with invalid Fund IDs: {len(invalid_fund_ids)}")

# Missing Investor for Account
print("\nMissing Investor for Account proportion:", investors_accounts_df['Investor for Account'].isnull().mean())

# ---------------------------------------------------------
# 3. managed_funds_df checks
print("\nChecking managed_funds_df")

# Fund Legal Name vs Fund ID
fund_name_id_diff = managed_funds_df[managed_funds_df['Fund Legal Name'] != managed_funds_df['Fund ID']]
print(f"\nFund Legal Name different from Fund ID: {len(fund_name_id_diff)} rows")

# Fund date consistency
date_issue = managed_funds_df[
    (managed_funds_df['Fund Start Date'] > managed_funds_df['Fund Close Date']) |
    (managed_funds_df['Fund Close Date'] > managed_funds_df['Fund Termination Date'])
]
print(f"Funds with date inconsistencies: {len(date_issue)}")

# Duplicate Fund IDs
print("\nDuplicate Fund IDs:", managed_funds_df['Fund ID'].duplicated().sum())

# ---------------------------------------------------------
# 4. transactions_with_details_df checks
print("\nChecking transactions_with_details_df")

# Entry Date parsing
transactions_with_details_df['Entry Date Parsed'] = pd.to_datetime(transactions_with_details_df['Entry Date'], errors='coerce')
print("Invalid Entry Dates:", transactions_with_details_df['Entry Date Parsed'].isnull().sum())

# Fund names validity
missing_funds = transactions_with_details_df[~transactions_with_details_df['Fund name'].isin(managed_funds_df['Fund ID'])]
print(f"Transactions with Fund names not in managed_funds_df: {len(missing_funds)}")

# Negative debit or credit
neg_debit = transactions_with_details_df[transactions_with_details_df['Entry Currency Debit'] < 0]
neg_credit = transactions_with_details_df[transactions_with_details_df['Entry Currency Credit'] < 0]
print(f"Negative debit rows: {len(neg_debit)}, Negative credit rows: {len(neg_credit)}")

# 'Unnamed' columns usage
unnamed_cols = [col for col in transactions_with_details_df.columns if 'Unnamed' in col]
print("\nNon-null counts in 'Unnamed' columns:")
print(transactions_with_details_df[unnamed_cols].notnull().sum())

# ---------------------------------------------------------
# 5. fund_ops_df checks
print("\nChecking fund_ops_df")

# Fund for Account null proportion
print("Fund for Account missing proportion:", fund_ops_df['Fund for Account'].isnull().mean())

# Investor for Account null proportion
print("Investor for Account missing proportion:", fund_ops_df['Investor for Account'].isnull().mean())

# Fund Op Date range
print("Fund Op Date range:", fund_ops_df['Fund Op Date'].min(), "to", fund_ops_df['Fund Op Date'].max())

# Negative Fund Currency Amounts
neg_fund_currency = fund_ops_df[fund_ops_df['Fund Currency Amount'] < 0]
print(f"Negative Fund Currency Amount rows: {len(neg_fund_currency)}")

# ---------------------------------------------------------
# 6. Cross-table integrity checks
print("\nCross-table referential integrity checks")

# Investor IDs in Accounts exist in Investors
invalid_investors = investors_accounts_df[~investors_accounts_df['Investor for Account'].isin(investors_df['Investor ID'])]
print(f"Accounts with invalid Investor IDs: {len(invalid_investors)}")

# Investor Accounts in Fund Ops exist in Investor Accounts
invalid_accounts = fund_ops_df[~fund_ops_df['Investor Account'].isin(investors_accounts_df['Investor Account ID'])]
print(f"Fund Ops with invalid Investor Accounts: {len(invalid_accounts)}")

# ---------------------------------------------------------
print("\n=== Data Validation Checks Complete ===")

=== Starting Data Validation Checks ===

Checking investors_df

Investor Region value counts:
Investor Region
???    4603
Name: count, dtype: int64

Investor Legal Name different from Investor ID: 0 rows

Missing Investor First Name proportion: 0.8416250271562025
Missing Investor Last Name proportion: 0.8416250271562025

Duplicate Investor IDs: 13

Checking investors_accounts_df

Duplicate Investor Account IDs: 0
Accounts with invalid Fund IDs: 0

Missing Investor for Account proportion: 0.020676979629345994

Checking managed_funds_df

Fund Legal Name different from Fund ID: 0 rows
Funds with date inconsistencies: 10

Duplicate Fund IDs: 0

Checking transactions_with_details_df
Invalid Entry Dates: 0
Transactions with Fund names not in managed_funds_df: 2424
Negative debit rows: 55, Negative credit rows: 14

Non-null counts in 'Unnamed' columns:
Unnamed: 25    0
Unnamed: 26    4
dtype: int64

Checking fund_ops_df
Fund for Account missing proportion: 0.9804007585639719
Investor for Acco